In [12]:
!pip install mecab-python3

In [13]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 1s (170 kB/s)
Reading package lists... Done


In [14]:
!apt-get install g++ openjdk-8-jdk python-dev python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
python3-dev is already the newest version (3.6.7-1~18.04).
openjdk-8-jdk is already the newest version (8u272-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [15]:
!pip3 install JPype1-py3

In [16]:
!pip3 install konlpy

In [17]:
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

In [18]:
# 이건 우선 패스할 것!
# 만약에 밑에까지 다 했는데, 오류 뜨면 이거 돌려보고 다시 해볼 것!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (167 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [19]:
!pip install konlpy

In [20]:
from konlpy.tag import Mecab
import numpy as np
import pickle
import re
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
cafe_data = pd.read_excel("/content/drive/My Drive/Colab Notebooks/데이터/F 카페(7,859).xlsx")
# cafe_data.head()

OSError: ignored

In [ ]:
# 단어 아니면 삭제
cafe_data['SENTENCE'] = cafe_data['SENTENCE'].str.replace("[^\w]|br", " ")

In [ ]:
pre_cafe_data = cafe_data.loc[0:6673,["SPEAKER","SENTENCE"]]

In [ ]:
# 점원과 고객이 한말 두개 따로 나누기.
# Q and A 형식으로 dataframe 만들기.
user = pre_cafe_data[pre_cafe_data["SPEAKER"] == "고객"]["SENTENCE"]
service = pre_cafe_data[pre_cafe_data["SPEAKER"] == "점원"]["SENTENCE"]

In [ ]:
user.name="user"
service.name="service"

In [ ]:
service = pd.DataFrame(service).reset_index(drop=True)
user = pd.DataFrame(user).reset_index(drop=True)
result_data = pd.concat([user, service], axis=1)
# result_data

In [ ]:
tokenizer = Mecab()

# yeswords = ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VX', 'VCP', 'VCN', 'MM', 'MAG', 'MAJ', 'IC']

encoder_input = []
for line in result_data["user"]:
    token = []
    words = tokenizer.pos(line)
    for word in words:
        # if word[1] in yeswords:
        token.append(word[0])
    encoder_input.append(token)

decoder_input = []
for line in result_data["service"]:
    token = []
    words = tokenizer.pos(line)
    for word in words:
        # if word[1] in yeswords:
        token.append(word[0])
    token.insert(0, "<start>")
    decoder_input.append(token)

decoder_output = []
for line in result_data["service"]:
    token = []
    words = tokenizer.pos(line)
    for word in words:
        # if word[1] in yeswords:
        token.append(word[0])
    token.append("<end>")
    decoder_output.append(token)
            

In [ ]:
tokenizer_text = Tokenizer()
tokenizer_text.fit_on_texts(encoder_input)
encoder_input = tokenizer_text.texts_to_sequences(encoder_input)

tokenizer_title = Tokenizer()
tokenizer_title.fit_on_texts(decoder_input)
tokenizer_title.fit_on_texts(decoder_output)
decoder_input = tokenizer_title.texts_to_sequences(decoder_input)
decoder_output = tokenizer_title.texts_to_sequences(decoder_output)

print(len(encoder_input))
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

In [ ]:
with open('tokenizer_text.pickle','wb') as fw:
    pickle.dump(tokenizer_text, fw)

In [ ]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_output = pad_sequences(decoder_output, padding="post")

In [ ]:
title_to_index = tokenizer_title.word_index
index_to_title = tokenizer_title.index_word

In [ ]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_output.shape)

print(len(encoder_input))
print(len(decoder_input))
print(len(decoder_output))

In [ ]:
with open('title_to_index.pickle','wb') as fw:
    pickle.dump(title_to_index, fw)
with open("index_to_title.pickle", "wb") as fw:
    pickle.dump(index_to_title, fw)

In [ ]:
test_size = 800
encoder_input_train = encoder_input[ :-test_size]
decoder_input_train = decoder_input[ :-test_size]
decoder_output_train = decoder_output[ :-test_size]

encoder_input_test = encoder_input[test_size: ]
decoder_input_test = decoder_input[test_size: ]
decoder_output_test = decoder_output[test_size: ]

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.models import Sequential
from gensim import models
from gensim.models.wrappers import FastText 
import gensim

In [ ]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/데이터/cc.ko.300.vec', encoding = 'ISO-8859-1')

In [ ]:
NUM_WORDS, EMB_DIM = word2vec_model.wv.vectors.shape

In [ ]:
encoder_inputs = Input(shape=(24,))
encoder_embed = Embedding(NUM_WORDS, EMB_DIM, trainable=False, weights=[word2vec_model.wv.vectors])(encoder_inputs)
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(EMB_DIM, return_sequences=True, return_state=True)(encoder_mask)

In [ ]:
decoder_inputs = Input(shape=(34,))
decoder_embed = Embedding(NUM_WORDS, EMB_DIM, trainable=False, weights=[word2vec_model.wv.vectors])(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)
decoder_lstm = LSTM(EMB_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])

In [ ]:
from attention import AttentionLayer
attn_layer = AttentionLayer()
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

decoder_concat_input = Concatenate()([decoder_outputs, attn_out])

decoder_dense = Dense(NUM_WORDS, activation= 'softmax')
decoder_softmax_outputs = decoder_dense(decoder_concat_input)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x=[encoder_input_train, decoder_input_train],
          y=decoder_output_train,
          callbacks=[es],
          validation_data=([encoder_input_test, decoder_input_test], decoder_output_test),
          batch_size=128, epochs=150)

In [ ]:
encoder_model = Model(encoder_inputs, [encoder_outputs, h_state, c_state])

encoder_h_state = Input(shape=(EMB_DIM,))
encoder_c_state = Input(shape=(EMB_DIM,))

pd_decoder_outputs,pd_h_state,pd_c_state = decoder_lstm(decoder_mask,
                                                        initial_state=[encoder_h_state,encoder_c_state])

pd_encoder_outputs = Input(shape=(24,EMB_DIM))
pd_attn_out, pd_attn_states = attn_layer([pd_encoder_outputs, pd_decoder_outputs])
pd_decoder_concat = Concatenate()([pd_decoder_outputs, pd_attn_out])

pd_decoder_softmax_outputs = decoder_dense(pd_decoder_concat)

decoder_model = Model([decoder_inputs, pd_encoder_outputs, encoder_h_state, encoder_c_state],
                      [pd_decoder_softmax_outputs, pd_h_state, pd_c_state])

In [ ]:
with open('encoder_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('encoder_model_weights.h5')

with open('decoder_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('decoder_model_weights.h5')

In [ ]:
input_stc = input()

token_stc = []
for line in tokenizer.pos(input_stc):
    # if line[1] in yeswords:
    token_stc.append(line[0])

# token_stc = input_stc.split()
encode_stc = tokenizer_text.texts_to_sequences([token_stc])
pad_stc = pad_sequences(encode_stc, maxlen=34, padding='post')

en_out, en_hidden, en_cell = encoder_model.predict(pad_stc)

predicted_seq = np.zeros((1,1))
predicted_seq[0,0] = title_to_index["<start>"]

decoded_stc = []

while True:
    output_words, h, c = decoder_model.predict([predicted_seq, en_out, en_hidden, en_cell])
    predicted_word = index_to_title[np.argmax(output_words[0,0])]
    
    if predicted_word == "<end>":
        break
    
    decoded_stc.append(predicted_word)

    predicted_seq = np.zeros((1,1))
    predicted_seq[0, 0] = np.argmax(output_words[0,0])

    en_hidden = h
    en_cell = c

print(" ".join(decoded_stc))